<a href="https://colab.research.google.com/github/cesar-claros/synergistic/blob/SocialAds/CIFAR10_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR 10

In [ ]:
! git clone https://github.com/cesar-claros/synergistic
% cd synergistic/

## Dependencies

In [ ]:
#%%
# Command line instalation
# ---------------------------
!pip install torch
!pip install gpytorch
!pip install tensorflow-determinism

# Imports
# ---------------------------
import io #Used as buffer
import sys
import matplotlib
import tensorflow as tf # Keras model for MNIST 
# matplotlib.use('qt5Agg')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import auxfunc.funcs as sgn
import auxfunc.trustscore as trs
import seaborn as sns
import torch
from scipy.stats import entropy, spearmanr
from sklearn import model_selection, svm, ensemble, linear_model, pipeline, metrics,\
      tree, neighbors, discriminant_analysis, gaussian_process, preprocessing, impute, decomposition
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern
plt.style.use(['ggplot','style/style.mplstyle'])
import os
import random

## Auxiliar Functions

In [3]:
#%%
# Define classic MLP architecture
def CNN_model(input_dim):
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(32,32,3))) 
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model

In [4]:
#%%
# Signaling function fitting and evaluation
def signalingFunction(X_train, y_train, y_train_pred_th, X_val, y_val, y_val_pred_th, X_test, y_test, y_test_pred_th, kernel='exponential', norm='l01', ex_dim=1):
    # X_train, X_val should be scaled. ex_dim=1 by default on sgn.signailing
    # Fit signaling function 
    exp = sgn.signaling(norm=norm) # idx = [train,test,val]
    exp.fit(X_train, y_train, y_train_pred_th, kernel=kernel, n_iter=500, lr=0.01, ex_dim=ex_dim)
    table_val = exp.evaluate(X_val, y_val, y_val_pred_th, rule_grid=np.linspace(0,3,30, endpoint=False), rho_grid=[0.1, 0.15])
    table_test = exp.test(X_test, y_test, y_test_pred_th, table_val['rule'].to_numpy(), table_val['eta'].to_numpy())
    table = pd.concat([table_val,table_test],axis=1)
    return table, exp

In [5]:
#%%
# Initialize model
def init_model(input_dim):
    # svm = False
    # model = MLP_model(input_dim=Data_X.shape[1], svm_obj=svm)
    model  = CNN_model(input_dim=input_dim)
    loss   = tf.keras.losses.categorical_crossentropy
    metric = ['accuracy']
    model.compile(loss=loss,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, 
                                                     beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                                     name='Adam'),
                  metrics=metric)
    return model

In [6]:
#%%
# Soft and thresholded output predictions
def pred_output(model, X):
    y_pred_soft = model.predict(X)
    y_pred_th   = np.argmax(y_pred_soft, axis=1)
    return y_pred_soft, y_pred_th

In [7]:
#%%
# Jaccard similarity index
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [8]:
#%%
# Baseline comparison
def baselineCriteria(y_val, y_val_pred_soft, y_val_pred_th, y_test, y_test_pred_soft, y_test_pred_th, table, exp,trust_val, trust_test):
      direction = 'further'
      # p_val = np.concatenate(y_val_pred_soft,axis=1)
      crit_val   = entropy(y_val_pred_soft, axis=1, base=10)
      # p_test = np.concatenate(y_test_pred_soft,axis=1)
      crit_test  = entropy(y_test_pred_soft, axis=1, base=10)
      # Criteria 1
      critFunc   = sgn.critEvaluation(norm='l01',direction=direction)
      d_val      = critFunc.evaluate(y_val, y_val_pred_th, crit_val, rho_grid=[0.1, 0.15])
      d_test     = critFunc.test(y_test, y_test_pred_th, crit_test, d_val['thresh'].to_numpy())
      crit_table = pd.concat([d_val,d_test],axis=1)
      # Best rules from signailing function on val are used to get UCBs on test
      gamma      = table['rule'].to_numpy().reshape(-1,1)
      f_test     = exp.gpr_mean_test + gamma*np.sqrt(exp.gpr_var_test)
      # Threshold values on val data. UCB Signailing (eta) and new criteria (theta) 
      eta        = table['eta'].to_numpy().reshape(-1,1)
      theta      = crit_table['thresh'].to_numpy().reshape(-1,1)
      if direction == 'closer':
        crit_mask, crit_idx = np.nonzero(crit_test.reshape(1,-1)<theta)
      else:
        crit_mask, crit_idx = np.nonzero(crit_test.reshape(1,-1)>theta)
      f_mask, f_idx = np.nonzero(f_test>eta)      
      shared = set(list(np.unique(f_mask))).intersection(set(list(np.unique(crit_mask))))
      # Jaccard index btw signaled instances using both methods for ith rule-threshold
      J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) if i in shared else np.nan for i in range(f_test.shape[0])]
      crit_table['jaccard']=J
      Sp = [spearmanr(f_test[i,:],crit_test)[0] for i in range(f_test.shape[0])]
      crit_table['spearman'] = Sp
      crit_table['gamma']    = gamma

      # Criteria 2
      critFuncSc = sgn.critEvaluation(norm='l01',direction='closer')
      s_val      = critFuncSc.evaluate(y_val, y_val_pred_th, trust_val, rho_grid=[0.1, 0.15])
      s_test     = critFuncSc.test(y_test, y_test_pred_th, trust_test, s_val['thresh'].to_numpy())
      score_table= pd.concat([s_val,s_test],axis=1)
      # Threshold values on val data. TrustScore (theta0) 
      theta0     = score_table['thresh'].to_numpy().reshape(-1,1) 
      crit_mask0, crit_idx0 = np.nonzero(trust_test.reshape(1,-1)<theta0)
      # Jaccard index btw signaled instances using both methods for ith rule-threshold  
      shared0    = set(list(np.unique(f_mask))).intersection(set(list(np.unique(crit_mask0))))
      J0         = [jaccard_similarity(crit_idx0[crit_mask0==i],f_idx[f_mask==i]) if i in shared0 else np.nan for i in range(f_test.shape[0])]
      Sp0        = [spearmanr(f_test[i,:],trust_test)[0] for i in range(f_test.shape[0])]
      score_table['jaccard']  = J0 
      score_table['spearman'] = Sp0  

      return crit_table,score_table,crit_test

## Signailing function and baselines

In [9]:
# For reproducibility
os.environ['TF_DETERMINISTIC_OPS'] = '1'
SEED = 12345
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [10]:
# %%
# INITIALIZATION
# ==============
# EXPERIMENT SETUP
# ================
# Load data set
(Data_X, Data_y), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# y, y_test = y.astype('int8'), y_test.astype('int8')
# Rescale the images from [0,255] to the [0.0,1.0] range.
Data_X, X_test = Data_X[...]/255.0, X_test[...]/255.0
n_cat  = np.unique(np.concatenate([Data_y,y_test]), return_counts=False)
n_cat  = n_cat.size
print("Number of original training examples:", len(Data_X))
print("Number of original testing examples:", len(X_test))
print("---------------")
# Subsample data (30% of original data both in training and testing)
Data_X, Data_X_sep, Data_y, Data_y_sep = model_selection.train_test_split(Data_X, Data_y, stratify=Data_y, test_size=0.75, random_state=SEED)
X_test, X_test_sep, y_test, y_test_sep = model_selection.train_test_split(X_test, y_test, stratify=y_test, test_size=0.75, random_state=SEED)
print("Number of subsampled training examples:", len(Data_X))
print("Number of subsampled testing examples:", len(X_test))

170508288/170498071 [==============================] - 13s 0us/step
Number of original training examples: 50000
Number of original testing examples: 10000
---------------
Number of subsampled training examples: 12500
Number of subsampled testing examples: 2500


In [11]:
#%%
# Assign labels
report_table    = []
trust_criteria  = []
report_criteria = []
report_plot     = []
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
clf = 'softmax_act'
addPredictions = True
accuracy = 0
fold     = 1
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', verbose=1)
y_test = tf.keras.utils.to_categorical(y_test)
for train, val in kf.split(Data_X, Data_y):
    X_train = Data_X[train]
    y_train = Data_y[train]
    y_train = tf.keras.utils.to_categorical(y_train)
    X_val   = Data_X[val]
    y_val   = Data_y[val]
    y_val   = tf.keras.utils.to_categorical(y_val)

    # ML Model fit & prediction
    model = init_model(input_dim=Data_X.shape[1:])
    # model.fit(X_train, y_train, batch_size=128, epochs=30, verbose=0, validation_data=(X_val, y_val), callbacks=[es])
    model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=0, validation_data=(X_val, y_val))
    y_train_pred_soft, y_train_pred_th = pred_output(model, X_train) # y_train_pred_soft&y_train_pred_th: 10-d vector  
    y_val_pred_soft, y_val_pred_th     = pred_output(model, X_val)   # y_val_pred_soft&y_val_pred_th    : 10-d vector
    y_test_pred_soft, y_test_pred_th   = pred_output(model, X_test)  # y_test_pred_soft&y_test_pred_th  : 10-d vector
    print('accuracy(Train)={}'.format(np.sum(y_train_pred_th==np.argmax(y_train,axis=1))/y_train_pred_th.size))

    # layer_outputs = [layer.output for layer in model.layers[4]]
    if clf == 'softmax_act':
            activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.layers[5].output) # Early feture extraction: 128 dense vector
            X_train_GP       = activation_model.predict(X_train)
            X_val_GP         = activation_model.predict(X_val)
            X_test_GP        = activation_model.predict(X_test)
    elif clf == 'softmax':
            X_train_GP       = X_train.reshape(-1,np.prod(X_train.shape[1:]))
            X_val_GP         = X_val.reshape(-1,np.prod(X_val.shape[1:]))
            X_test_GP        = X_test.reshape(-1,np.prod(X_test.shape[1:]))

    if addPredictions:
            # Add predictions
            X_train_GP = np.concatenate((X_train_GP, y_train_pred_soft), axis=1)
            X_val_GP   = np.concatenate((X_val_GP, y_val_pred_soft), axis=1)
            X_test_GP  = np.concatenate((X_test_GP, y_test_pred_soft), axis=1)
    scaleX_GP  = preprocessing.StandardScaler().fit(X_train_GP)
    X_train_GP = scaleX_GP.transform(X_train_GP)
    X_val_GP   = scaleX_GP.transform(X_val_GP)
    X_test_GP  = scaleX_GP.transform(X_test_GP)

    # Signailing function. Call
    table, exp = signalingFunction(X_train_GP, np.argmax(y_train, axis=1), y_train_pred_th, \
                                   X_val_GP, np.argmax(y_val, axis=1), y_val_pred_th,\
                                   X_test_GP, np.argmax(y_test, axis=1), y_test_pred_th,\
                                   kernel='e*e', ex_dim=y_train_pred_soft.shape[1])

    report_table.append(pd.concat([pd.DataFrame({'fold':[fold]*table.shape[0]}),table],axis=1))

    # Trust Score fitted on train data to evaluate loss reduction on val-test data
    trust_model = trs.TrustScore()
    trust_model.fit(X=X_train_GP,Y=y_train,classes=n_cat)
    trust_val,class_val   = trust_model.score(X_val_GP, y_val_pred_th)
    trust_test,class_test = trust_model.score(X_test_GP, y_test_pred_th)
    
    # Baseline for comparison
    crit_table,score_table,crit_test = baselineCriteria(np.argmax(y_val, axis=1), y_val_pred_soft, y_val_pred_th,\
                                                        np.argmax(y_test, axis=1), y_test_pred_soft, y_test_pred_th,\
                                                        table, exp,\
                                                        trust_val, trust_test)
   
    report_criteria.append(pd.concat([pd.DataFrame({'fold':[fold]*crit_table.shape[0]}),crit_table],axis=1))
    trust_criteria.append(pd.concat([pd.DataFrame({'fold':[fold]*score_table.shape[0]}),score_table],axis=1))

    score = np.sum(np.argmax(y_val, axis=1)==y_val_pred_th)/np.size(np.argmax(y_val, axis=1))
    if accuracy < score:
      accuracy = score
      table_best = table
      crit_table_best = crit_table
      exp_best = exp
      y_test_best = y_test
      y_test_pred_soft_best = y_test_pred_soft
      y_test_pred_th_best = y_test_pred_th
      X_test_best = X_test
    fold +=1
    del(model)

accuracy(Train)=0.7681
initializing cuda...
lr=0.01, n_iterations=500


/usr/local/lib/python3.7/dist-packages/gpytorch/utils/linear_cg.py:234: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [1, 11].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  torch.sum(mul_storage, -2, keepdim=True, out=alpha)


Iter 491/500 - Loss: 0.389  noise: 0.072
Iter 492/500 - Loss: 0.389  noise: 0.072
Iter 493/500 - Loss: 0.389  noise: 0.071
Iter 494/500 - Loss: 0.390  noise: 0.071
Iter 495/500 - Loss: 0.389  noise: 0.071
Iter 496/500 - Loss: 0.389  noise: 0.071
Iter 497/500 - Loss: 0.389  noise: 0.071
Iter 498/500 - Loss: 0.389  noise: 0.071
Iter 499/500 - Loss: 0.389  noise: 0.071
Iter 500/500 - Loss: 0.388  noise: 0.071
evaluating with cuda...
evaluating with cuda...
accuracy(Train)=0.7254
initializing cuda...
lr=0.01, n_iterations=500


/usr/local/lib/python3.7/dist-packages/gpytorch/utils/linear_cg.py:234: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [1, 11].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  torch.sum(mul_storage, -2, keepdim=True, out=alpha)


Iter 491/500 - Loss: 0.470  noise: 0.091
Iter 492/500 - Loss: 0.469  noise: 0.091
Iter 493/500 - Loss: 0.470  noise: 0.091
Iter 494/500 - Loss: 0.470  noise: 0.091
Iter 495/500 - Loss: 0.470  noise: 0.091
Iter 496/500 - Loss: 0.471  noise: 0.091
Iter 497/500 - Loss: 0.471  noise: 0.091
Iter 498/500 - Loss: 0.470  noise: 0.091
Iter 499/500 - Loss: 0.470  noise: 0.090
Iter 500/500 - Loss: 0.471  noise: 0.090
evaluating with cuda...
evaluating with cuda...
accuracy(Train)=0.7724
initializing cuda...
lr=0.01, n_iterations=500


...

Iter 491/500 - Loss: 0.398  noise: 0.077
Iter 492/500 - Loss: 0.399  noise: 0.077
Iter 493/500 - Loss: 0.398  noise: 0.077
Iter 494/500 - Loss: 0.398  noise: 0.077
Iter 495/500 - Loss: 0.399  noise: 0.077
Iter 496/500 - Loss: 0.398  noise: 0.077
Iter 497/500 - Loss: 0.399  noise: 0.077
Iter 498/500 - Loss: 0.399  noise: 0.076
Iter 499/500 - Loss: 0.398  noise: 0.076
Iter 500/500 - Loss: 0.398  noise: 0.076
evaluating with cuda...
evaluating with cuda...
accuracy(Train)=0.7582
initializing cuda...
lr=0.01, n_iterations=500


/usr/local/lib/python3.7/dist-packages/gpytorch/utils/linear_cg.py:234: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [1, 11].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  torch.sum(mul_storage, -2, keepdim=True, out=alpha)


Iter 491/500 - Loss: 0.408  noise: 0.076
Iter 492/500 - Loss: 0.408  noise: 0.076
Iter 493/500 - Loss: 0.408  noise: 0.076
Iter 494/500 - Loss: 0.408  noise: 0.076
Iter 495/500 - Loss: 0.408  noise: 0.076
Iter 496/500 - Loss: 0.406  noise: 0.076
Iter 497/500 - Loss: 0.408  noise: 0.076
Iter 498/500 - Loss: 0.408  noise: 0.076
Iter 499/500 - Loss: 0.408  noise: 0.076
Iter 500/500 - Loss: 0.407  noise: 0.076
evaluating with cuda...
evaluating with cuda...
accuracy(Train)=0.7622
initializing cuda...
lr=0.01, n_iterations=500


/usr/local/lib/python3.7/dist-packages/gpytorch/utils/linear_cg.py:234: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [1, 11].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  torch.sum(mul_storage, -2, keepdim=True, out=alpha)


Iter 491/500 - Loss: 0.413  noise: 0.077
Iter 492/500 - Loss: 0.414  noise: 0.077
Iter 493/500 - Loss: 0.413  noise: 0.077
Iter 494/500 - Loss: 0.414  noise: 0.077
Iter 495/500 - Loss: 0.412  noise: 0.076
Iter 496/500 - Loss: 0.413  noise: 0.076
Iter 497/500 - Loss: 0.414  noise: 0.076
Iter 498/500 - Loss: 0.413  noise: 0.076
Iter 499/500 - Loss: 0.414  noise: 0.076
Iter 500/500 - Loss: 0.412  noise: 0.076
evaluating with cuda...
evaluating with cuda...


## Summary

In [12]:
report_table_concat = pd.concat(report_table)
report_table_concat = report_table_concat.groupby('rho_user').agg({"rho_hat_test":[np.median],"%loss_red_test": [np.median,min, max]})
report_table_concat

rho_hat_test %loss_red_test              
               median         median    min    max
rho_user                                          
0.10             0.10          18.07  15.51  19.49
0.15             0.15          25.84  25.07  29.35

In [13]:
report_criteria_concat = pd.concat(report_criteria)
report_criteria_concat = report_criteria_concat.groupby('rho_user').agg({"rho_hat_test":[np.median],"%loss_red_test":[np.median,min, max],"jaccard":[np.median,min, max]})
report_criteria_concat

rho_hat_test %loss_red_test         ...   jaccard                    
               median         median    min  ...    median       min       max
rho_user                                     ...                              
0.10             0.11          18.93  16.75  ...  0.257709  0.246445  0.333333
0.15             0.15          25.93  24.87  ...  0.351449  0.301325  0.390476

[2 rows x 7 columns]

In [14]:
report_trust_concat    = pd.concat(trust_criteria)
report_trust_concat    = report_trust_concat.groupby('rho_user').agg({"rho_hat_test":[np.median],"%loss_red_test":[np.median,min, max],"jaccard":[np.median,min, max]})
report_trust_concat

rho_hat_test %loss_red_test         ...   jaccard                    
               median         median    min  ...    median       min       max
rho_user                                     ...                              
0.10             0.09          15.84  14.24  ...  0.238462  0.195000  0.263027
0.15             0.14          24.58  20.91  ...  0.299145  0.238532  0.341969

[2 rows x 7 columns]

In [ ]:
report_table_concat = pd.concat(report_table)
# report_table_concat
report_criteria_concat = pd.concat(report_criteria)
# report_criteria_concat
# p-value column
p_value_col = report_table_concat['p_value'] #Add
cols_CQT = ['rho_user','corrected_test','queries_test','total_wrong_test','loss_query_test']
cols_rholoss = ['rho_user','rho_hat_test','%loss_red_test']

# Dataframes for f(x)
df_fx_CQT = pd.DataFrame(report_table_concat[cols_CQT])
df_fx_rholoss = pd.DataFrame(report_table_concat[cols_rholoss])
# results_fx_CQT = df_fx_CQT.loc[p_value_col <= 0.05].copy()
results_fx_rholoss = df_fx_rholoss.loc[p_value_col <= 0.05].copy()
results_fx_CQT = df_fx_CQT.copy()
# results_fx_rholoss = df_fx_rholoss.copy()

results_fx_rholoss_bri = results_fx_rholoss.groupby(results_fx_rholoss.index)
fx_rholoss_median = results_fx_rholoss_bri.median()
fx_rholoss_q1 = results_fx_rholoss_bri.quantile(q=0)
fx_rholoss_q3 = results_fx_rholoss_bri.quantile(q=1)
results_fx_CQT_bri = results_fx_CQT.groupby(results_fx_CQT.index)
fx_CQT_median = results_fx_CQT_bri.median()
fx_CQT_q1 = results_fx_CQT_bri.quantile(q=0)
fx_CQT_q3 = results_fx_CQT_bri.quantile(q=1)

# Dataframes for g(x)
df_gx_CQT = pd.DataFrame(report_criteria_concat[cols_CQT])
df_gx_rholoss = pd.DataFrame(report_criteria_concat[cols_rholoss])
# results_gx_CQT = df_gx_CQT.loc[p_value_col <= 0.05].copy()
results_gx_rholoss = df_gx_rholoss.loc[p_value_col <= 0.05].copy()
results_gx_CQT = df_gx_CQT.copy()
# results_gx_rholoss = df_gx_rholoss.copy()

results_gx_rholoss_bri = results_gx_rholoss.groupby(results_gx_rholoss.index)
gx_rholoss_median = results_gx_rholoss_bri.median()
gx_rholoss_q1 = results_gx_rholoss_bri.quantile(q=0)
gx_rholoss_q3 = results_gx_rholoss_bri.quantile(q=1)
results_gx_CQT_bri = results_gx_CQT.groupby(results_gx_CQT.index)
gx_CQT_median = results_gx_CQT_bri.median()
gx_CQT_q1 = results_gx_CQT_bri.quantile(q=0)
gx_CQT_q3 = results_gx_CQT_bri.quantile(q=1)

# Signaling function statistics (median(q1-q3)) LaTex
output_test = io.StringIO()
# numRows = fx_median.shape[0]
# numCols = fx_median.shape[1]
output_test.write("results_test (dataset|method|(Q,C,T)|\hat{rho_test}|%loss_red_test\n")
output_test.write("----------\n")

for rho in [0.10,0.15]:
  # output_test.write("rho={:.2f}\\\\\n".format(rho))

  fx_CQT_filtered = results_fx_CQT.loc[results_fx_CQT['rho_user']==rho]
  gx_CQT_filtered = results_gx_CQT.loc[results_gx_CQT['rho_user']==rho]
  p_value_filtered = p_value_col[results_fx_CQT['rho_user']==rho]
  # print(p_value_filtered)
  n_folds = fx_CQT_filtered.shape[0]
  row_fx = [' ', ' ',' ',r'{:.2f}'.format(rho),r'$f(x)$']
  row_gx = [' ',' ',' ',' ',r'$g(x)$']

  row_fx_CQT = [r'({:.0f},{:.0f},{:.0f})'.format(val1,val2,val3) for val1,val2,val3 in zip(fx_CQT_median.loc[fx_CQT_median['rho_user']==rho,'queries_test'],\
                                                      fx_CQT_median.loc[fx_CQT_q3['rho_user']==rho,'corrected_test'],\
                                                      fx_CQT_median.loc[fx_CQT_q1['rho_user']==rho,'total_wrong_test'])]
  row_fx_rho = [r'{:.2f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_fx_loss = [r'{:.1f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_fx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(fx_CQT_median.loc[fx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q3.loc[fx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q1.loc[fx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_fx = row_fx + row_fx_CQT + row_fx_rho + row_fx_loss + row_fx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_fx)))

  row_gx_CQT = [r'({:.0f},{:.0f},{:.0f})'.format(val1,val2,val3) for val1,val2,val3 in zip(gx_CQT_median.loc[gx_CQT_median['rho_user']==rho,'queries_test'],\
                                                      gx_CQT_median.loc[gx_CQT_q3['rho_user']==rho,'corrected_test'],\
                                                      gx_CQT_median.loc[gx_CQT_q1['rho_user']==rho,'total_wrong_test'])]
  row_gx_rho = [r'{:.2f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_gx_loss = [r'{:.1f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_gx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(gx_CQT_median.loc[gx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q3.loc[gx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q1.loc[gx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_gx = row_gx + row_gx_CQT + row_gx_rho + row_gx_loss + row_gx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_gx)))
  output_test.write("\\cline{4-9}\n")


print(output_test.getvalue())

results_test (dataset|method|(Q,C,T)|\hat{rho_test}|%loss_red_test
----------
  &   &   & 0.10 & $f(x)$ & (242,186,1021) & 0.10 & 17.6 & 0.74(0.77-0.72)\\
  &   &   &   & $g(x)$ & (264,190,1021) & 0.11 & 18.6 & 0.71(0.72-0.65)\\
\cline{4-9}
  &   &   & 0.15 & $f(x)$ & (386,281,1021) & 0.15 & 27.5 & 0.71(0.75-0.69)\\
  &   &   &   & $g(x)$ & (383,263,1021) & 0.15 & 25.2 & 0.69(0.69-0.63)\\
\cline{4-9}



In [ ]:
##%
# Boxplot (loss reduction in test set)
report_table_concat = pd.concat(report_table)
cols_table = ['p_value','rho_user','%reduction_test']
df_boxplot_table = pd.DataFrame(report_table_concat[cols_table])
df_boxplot_table['label'] = df_boxplot_table.shape[0]*['$f(x)$']
report_criteria_concat = pd.concat(report_criteria)
columns_crit = ['rho_user','%reduction_test']
df_boxplot_crit = pd.DataFrame(report_criteria_concat[columns_crit])
df_boxplot_crit['label'] = df_boxplot_crit.shape[0]*['$g(x)$']
# p-value median
p_value_col = df_boxplot_table['p_value'] #Add
p_value_by_row_index = df_boxplot_table['p_value'].groupby(df_boxplot_table.index)
p_value_median = p_value_by_row_index.median()
# Boxplot (jaccard index in test set)
columns_jac = ['rho_user','jaccard']
df_jaccard = pd.DataFrame(report_criteria_concat[columns_jac])
# Unfiltered Result dataframes
cols_fx = ['rho_user','%reduction_val','budget','%reduction_test']
results_fx = pd.DataFrame(report_table_concat[cols_fx])
cols_fxgx = ['rho_user','%reduction_test', 'jaccard']
results_fxgx = pd.concat([df_boxplot_table[cols_fxgx[:2]], df_boxplot_crit[cols_fxgx[1]], df_jaccard[cols_fxgx[2]]], axis=1)
# Filter experiments with p_value > 0.05
df_boxplot_crit = df_boxplot_crit.loc[df_boxplot_table['p_value'] <= 0.05]
df_jaccard = df_jaccard.loc[df_boxplot_table['p_value'] <= 0.05]
df_boxplot_table = df_boxplot_table.loc[df_boxplot_table['p_value'] <= 0.05]
# Boxplot with filtered values only
frames = [df_boxplot_table, df_boxplot_crit]
df = pd.concat(frames)

In [ ]:
# Avoid plotting when median(p_value)>0.5
for i in range(p_value_median.shape[0]):
    if p_value_median.iloc[i]>0.05:
      df.loc[df.index==i,'%reduction_test'] = np.nan
      df_jaccard.loc[df_jaccard.index==i, 'jaccard'] = np.nan

In [ ]:
# Dataframe for results f(x)
results_fx = results_fx.loc[p_value_col <= 0.05].copy()
results_fx_by_row_index = results_fx.groupby(results_fx.index)
fx_median = results_fx_by_row_index.median()
fx_q1 = results_fx_by_row_index.quantile(q=0.25)
fx_q3 = results_fx_by_row_index.quantile(q=0.75)
# Signaling function statistics (median(q1-q3)) LaTex
output_fx = io.StringIO()
numRows = fx_median.shape[0]
numCols = fx_median.shape[1]
output_fx.write("results_fx (\\rho|%reduction_val|sig_rate|%reduction_test|H0)\n")
output_fx.write("----------\n")
for i in range(numRows):
  row = [r'{:.2f}'.format(val1) if p_value_median[i]>0.05 and j==0 else r'{}' if p_value_median[i]>0.05 and j!=0\
         else r'{:.2f}'.format(val1) if (j==0) else r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) if (j==2)\
         else r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) for val1,val2,val3,j in zip(fx_median.iloc[i],fx_q1.iloc[i],fx_q3.iloc[i],range(numCols))]
  output_fx.write("{{}} & {{}} & %s & {H0} \\\\\n".format(H0=r'$\surd$' if p_value_median[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output_fx.getvalue())

In [ ]:
# Dataframe for comparison f(x)-g(x)
results_fxgx = results_fxgx.loc[p_value_col <= 0.05].copy()
results_fxgx_by_row_index = results_fxgx.groupby(results_fxgx.index)
fxgx_median = results_fxgx_by_row_index.median()
fxgx_q1 = results_fxgx_by_row_index.quantile(q=0.25)
fxgx_q3 = results_fxgx_by_row_index.quantile(q=0.75)
# Baseline comparison statistics (median(q1-q3)) LaTex
output_fxgx = io.StringIO()
numRows = fxgx_median.shape[0]
numCols = fxgx_median.shape[1]
output_fxgx.write("results_fxgx (\\rho|%reduction_test(fx)|%reduction_test(fxgx)|Jaccard|H0\n")
output_fxgx.write("------------\n")
for i in range(numRows):
  row = [r'{:.2f}'.format(val1) if p_value_median[i]>0.05 and j==0 else r'{}' if p_value_median[i]>0.05 and j!=0\
         else r'{:.2f}'.format(val1) if (j==0) else r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) if (j==3)\
         else r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) for val1,val2,val3,j in zip(fxgx_median.iloc[i],fxgx_q1.iloc[i],fxgx_q3.iloc[i],range(numCols))]
  output_fxgx.write("{{}} & {{}} & %s & {H0} \\\\\n".format(H0=r'$\surd$' if p_value_median[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output_fxgx.getvalue())

In [ ]:
#%%
# Save results in csv fomat
path_csv = "drive/My Drive/NIPS2020/results/cifar10/results_{clf}_yhat{yhat}_pca{pca}.csv".format(clf=clf, pca=applyPCA, yhat=addPredictions)
results = pd.concat([results_fx, results_fxgx, p_value_col.loc[p_value_col <= 0.05]], keys=['fx', 'fxgx', ''], axis=1).to_csv(path_csv, index=True, header=True)
# modified output
# Save results in tex fomat
L = [output_fx.getvalue(),output_fxgx.getvalue()]
path_txt = "drive/My Drive/NIPS2020/results/cifar10/results_{clf}_yhat{yhat}_pca{pca}.txt".format(clf=clf, pca=applyPCA, yhat=addPredictions)
txt = open(path_txt, "w") 
txt.writelines(L) 
txt.close() #to change file access modes

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15, 5.1), constrained_layout=False, dpi=90)
pal = sns.color_palette('Paired')
sns.boxplot(x=df['rho_user'], y=df['%reduction_test'], hue='label', data=df, ax=ax[0], palette=pal)
ax[0].set_xlabel(r'budget $\rho$')
ax[0].set_ylabel(r'Loss reduction $r_{test}(\%)$')
ax[0].legend(loc='upper left')
pal = sns.color_palette('BuGn_r')
sns.boxplot(x=df_jaccard['rho_user'], y=df_jaccard['jaccard'], data=df_jaccard, ax=ax[1], palette=pal)
ax[1].set_xlabel(r'budget $\rho$')
ax[1].set_ylabel(r'Jaccard index $J$')
plt.tight_layout()
path_fig_fxgx = "drive/My Drive/NIPS2020/results/cifar10/fig_fxgx_{clf}_yhat{yhat}_pca{pca}.pdf".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fxgx, bbox_inches='tight', facecolor='w')

In [ ]:
rho = 0.15
X_test_best = X_test_best.squeeze()
rule = table_best.loc[table_best.rho_user == rho]['rule'].to_numpy()
eta = table_best.loc[table_best.rho_user == rho]['eta'].to_numpy()[0]
theta = crit_table_best.loc[crit_table_best.rho_user == rho]['thresh'].to_numpy()[0]
f_test = exp_best.gpr_mean_test+rule*np.sqrt(exp_best.gpr_var_test)
top_n = 5 # Top n selected instances in test set
top_f_idx = np.argpartition(f_test, -top_n)[-top_n:]
top_f_idx = top_f_idx[np.argsort(f_test[top_f_idx])[::-1]]# Added
# p_test = np.concatenate((y_test_pred_soft_best,1-y_test_pred_soft_best),axis=1)
crit_test = entropy(y_test_pred_soft_best, axis=1, base=10)
top_crit_idx = np.argpartition(crit_test, -top_n)[-top_n:]
top_crit_idx = top_crit_idx[np.argsort(crit_test[top_crit_idx])[::-1]]# Added

output_text = io.StringIO()
print('eta={:.3f},theta={:.3f}'.format(eta,theta))

eta=0.538,theta=0.722


In [ ]:
roc_f = metrics.roc_auc_score(exp_best.L_test, f_test)
roc_crit = metrics.roc_auc_score(exp_best.L_test, crit_test)

In [ ]:
# Plot selected instances
# initialize the label 
labelNames = ['airplane', 'auto', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# X_test_best = X_test.reshape(-1,28,28)
# y_hat_best = y_test_pred_th[top_f_idx]
# y_th_best = np.argmax(y_test[top_f_idx], axis=1)
# Plot instances
row,col = 2,top_n
fig1, ax = plt.subplots(row, col, figsize=(6.0, 3.5), constrained_layout=True, dpi=120)
# fig1.subplots_adjust(wspace=0.1, hspace=0.35, top=0.5)
# for i in top_f_idx:
j = 0
for i,k in zip(top_f_idx, top_crit_idx):
    if np.argmax(y_test_best, axis=1)[i] != y_test_pred_th_best[i]:
        color = 'red'
    else:
        color = 'green'
    # ax = fig1.add_subplot(3, 4, j+1)
    ax[0][j].imshow(X_test_best[i, :, :, :])
    ax[0][j].set_title(r'$y=${s1}'.format(s1=labelNames[np.argmax(y_test_best, axis=1)[i]])+',\n'+\
                    r'$\hat{{y}}=${s2}'.format(s2=labelNames[y_test_pred_th_best[i]]), color=color, fontsize=13)
    ax[0][j].set_xlabel(r"$g(x)=${:.2f}".format(crit_test[i])+'\n$f(x)=${:.2f}'.format(f_test[i]), fontsize=13)
    ax[0][j].set_xticks([])
    ax[0][j].set_yticks([])

    if np.argmax(y_test_best, axis=1)[k] != y_test_pred_th_best[k]:
        color = 'red'
    else:
        color = 'green'
    # ax = fig1.add_subplot(3, 4, j+1)
    ax[1][j].imshow(X_test_best[k, :, :, :])
    ax[1][j].set_title(r'$y=${s1}'.format(s1=labelNames[np.argmax(y_test_best, axis=1)[k]])+',\n'+\
                    r'$\hat{{y}}=${s2}'.format(s2=labelNames[y_test_pred_th_best[k]]), color=color, fontsize=13)
    ax[1][j].set_xlabel(r"$g(x)=${:.2f}".format(crit_test[k])+'\n$f(x)=${:.2f}'.format(f_test[k]), fontsize=13)
    ax[1][j].set_xticks([])
    ax[1][j].set_yticks([])
    j = j + 1
fig1.set_constrained_layout_pads(w_pad=0, h_pad=0, hspace=0.01, wspace=-.5)
ax[0][0].set_ylabel(r"$f(x)>\eta$"+"\n"+r"$(AUC={:.2f})$".format(roc_f), fontsize=13)
# ax[0][0].set_ylabel(r"$f(x)>\eta$", fontsize=14)
ax[1][0].set_ylabel(r"$g(x)>\theta$"+"\n"+r"$(AUC={:.2f})$".format(roc_crit), fontsize=13)
# ax[1][0].set_ylabel(r"$g(x)>\theta$", fontsize=14)

# fig1.text(0.5, 0.01, r'$\rho={},~|f(x)>\eta|={},~|g(x)>\theta|={}$'.format(rho,np.sum(f_test>eta),np.sum(crit_test>theta)), ha='center', fontsize = 12)
# plt.suptitle(r'Top {} selected instances'.format(top_n), fontsize=15)
# plt.tight_layout()
path_fig_fxgx_test = "drive/My Drive/NIPS2020/results/cifar10/fig_fxgx_test_{clf}_yhat{yhat}_pca{pca}.svg".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fxgx_test, bbox_inches='tight', facecolor='w')